# Stable Diffusion Inference & Generation Control (Local Version)

This notebook has been adapted for local execution, supporting both CPU and CUDA (GPU) devices.

In [1]:
import os
import torch
from diffusers import DiffusionPipeline
import matplotlib.pyplot as plt
from PIL import Image
from contextlib import nullcontext

# Device Configuration
if torch.cuda.is_available():
    device = "cuda"
    dtype = torch.float16
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    dtype = torch.float32
    print("Using CPU. Note: Generation will be slow.")

model_path = "./sd15_local"
model_id = "runwayml/stable-diffusion-v1-5"

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


Using GPU: Tesla T4


In [ ]:
# Load or Download Model
if not os.path.exists(model_path):
    print(f"Downloading model '{model_id}' ...")
    pipe = DiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=dtype
    )
    pipe.save_pretrained(model_path)
    print(f"Model saved to {model_path}")
else:
    print(f"Loading model from {model_path} ...")
    pipe = DiffusionPipeline.from_pretrained(
        model_path,
        torch_dtype=dtype
    )

pipe = pipe.to(device)
print("Stable Diffusion ready \ud83d\ude80")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/196 [00:00<?, ?it/s]

CLIPTextModel LOAD REPORT from: /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d2269ed5ad43b0592e9a14/text_encoder
Key                                | Status     |  | 
-----------------------------------+------------+--+-
text_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading weights:   0%|          | 0/396 [00:00<?, ?it/s]

StableDiffusionSafetyChecker LOAD REPORT from: /root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d2269ed5ad43b0592e9a14/safety_checker
Key                                               | Status     |  | 
--------------------------------------------------+------------+--+-
vision_model.vision_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
def generate_image(pipe, prompt, seed=None, negative_prompt=None, steps=25, cfg=7.5):
    if seed is None:
        seed = torch.seed()
    
    # Use a fixed generator for reproducible results if seed is provided
    generator = torch.Generator(device=device).manual_seed(seed)
    
    # Autocast only for CUDA
    amp_ctx = torch.autocast("cuda") if device == "cuda" else nullcontext()

    with torch.inference_mode(), amp_ctx:
        result = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=steps,
            guidance_scale=cfg,
            generator=generator,
        )
    return result.images[0]

def show_two_images(img1, title1, img2, title2):
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.imshow(img1)
    plt.title(title1)
    plt.axis("off")
    
    plt.subplot(1, 2, 2)
    plt.imshow(img2)
    plt.title(title2)
    plt.axis("off")
    
    plt.tight_layout()
    plt.show()

## 1. Comparison: Randomness (No Seed)

Generating two images with the same prompt but without a fixed seed results in different images.

In [ ]:
prompt = "A futuristic AI lab, holographic screens, ultra detailed"

print("Generating image 1...")
img1 = generate_image(pipe, prompt)
print("Generating image 2...")
img2 = generate_image(pipe, prompt)

show_two_images(img1, "Random Seed (Run 1)", img2, "Random Seed (Run 2)")

## 2. Comparison: Reproducibility (With Seed)

Providing the same seed ensures the exact same image is generated.

In [ ]:
seed = 42

print("Generating image 1 with fixed seed...")
img3 = generate_image(pipe, prompt, seed=seed)
print("Generating image 2 with fixed seed...")
img4 = generate_image(pipe, prompt, seed=seed)

show_two_images(img3, f"Fixed Seed {seed} (Run 1)", img4, f"Fixed Seed {seed} (Run 2)")

## 3. Influence of Negative Prompts

Negative prompts allow you to specify what you *don't* want to see in the image.

In [ ]:
prompt_nature = "A beautiful mountain landscape with a clear lake, oil painting style"
neg_prompt = "trees, forest, green color"
seed = 123

print("Generating image without negative prompt...")
img5 = generate_image(pipe, prompt_nature, seed=seed)
print("Generating image with negative prompt...")
img6 = generate_image(pipe, prompt_nature, seed=seed, negative_prompt=neg_prompt)

show_two_images(img5, "No Negative Prompt", img6, "With Negative Prompt (No trees)")

## 4. CFG Scale (Guidance Scale)

CFG scale controls how closely the model follows the prompt.

In [ ]:
prompt_cat = "A cute cat wearing a space suit, digital art"
seed = 777

print("Generating with CFG=2.0 (Lower guidance)...")
img7 = generate_image(pipe, prompt_cat, seed=seed, cfg=2.0)
print("Generating with CFG=15.0 (Higher guidance)...")
img8 = generate_image(pipe, prompt_cat, seed=seed, cfg=15.0)

show_two_images(img7, "Low CFG (2.0)", img8, "High CFG (15.0)")